# An interactive visual demonstration of Bayes theorem
P(A|B)=P(A,B)/P(B)

In [20]:
# Import required libraries
import plotly.plotly as py
import plotly.figure_factory as ff
import plotly
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objs as go

# Set plotly to offline
plotly.offline.init_notebook_mode(connected=True)

# Prepare the labels that will be displayed on the vertical axis
y = ['P(B|A)','P(A|B)','P(A,B)','P(B)','P(A)']

# Prepare chart styles
# The "invisible" style is used on the top left and top right to create blank space
invisibleStyle = dict(color = 'rgba(255, 255, 255, 0.0)',line = dict(color = 'rgba(255, 255, 255, 0.0)',width = 0))
# The "range" style is used to represent the probability space.
# For PA, PB and PAB, the probability space is the parent [0,1] space.
# For P(A|B), the probability space is B.
# For P(B|A), the probability space is A.
rangeStyle = dict(color = 'rgba(100, 100, 100, 0.7)',line = dict(color = 'rgba(50, 50, 50, 0)',width = 0))
# The "probability" style is used to represent the probability value.
probabilityStyle = dict(color = 'rgba(255, 50, 50, 0.8)',line = dict(color = 'rgba(180, 0, 0, 1.0)',width = 3))

# This function is called whenever the user plays with the widgets
def widgetInteraction(PA, PB, PAB):
    
    PAgivenB = PAB / PB
    PBgivenA = PAB / PA
    
    # Trace 0 is the out-of-range space on the extreme left.
    trace0 = go.Bar(
        y=y,
        x=[0, PA - PAB, 0, 0, 0],
        name='Out of range',
        orientation = 'h',
        marker = invisibleStyle
    )
    # Trace 1 is the probability space on the left.
    trace1 = go.Bar(
        y=y,
        x=[PA - PAB, 0, PA - PAB, PA - PAB, 0],
        name='Range',
        orientation = 'h',
        marker = rangeStyle
    )
    # Trace 2 is the probability.
    trace2 = go.Bar(
        y=y,
        x=[PAB, PAB, PAB, PB, PA],
        text=['{0:10.2f}'.format(PBgivenA), '{0:10.2f}'.format(PAgivenB), '{0:10.2f}'.format(PAB), '{0:10.2f}'.format(PB), '{0:10.2f}'.format(PA)],
        textposition = 'auto',
        name='Probability',
        orientation = 'h',
        marker = probabilityStyle
    )
    # Trace 3 is the probability space on the right.
    trace3 = go.Bar(
        y=y,
        x=[0, PB - PAB, 1 - PA, 1 - PB - PA + PAB, 1 - PA],
        name='Range',
        orientation = 'h',
        marker = rangeStyle
    )
    # We don't need a space 4 because on the extreme right, the graph will be blank anyway.

    data = [trace0, trace1, trace2, trace3]
    layout = go.Layout(
        barmode='stack'
    )

    # The widget UI for float numbers is not proportional to the min / max values.
    # Hence, unfortunately, the P(A,B) widget whose min/max range is constrained
    # and may be smaller than P(A) and P(B), the user will get the wrong impression
    # that pushing the widget to the top right will yield 1. To partly compensate
    # this unfortunate situation, I expressely display the min / max values.
    print('{0:10.2f} <= P(A,B) <= {1:10.2f}'.format(getPABmin(PA, PB),getPABmax(PA, PB)))

    # And then we can draw the plot.
    fig = go.Figure(data=data, layout=layout)
    plotly.offline.iplot(fig, filename='marker-h-bar')

# This function will be called whenever the value of the PA widget is changed
def onWidgetPAUpdate(change):
    PB = widgetPB.value
    PAB = widgetPAB.value
    PANew = change.get('new', widgetPA.value)
    PAOld = change.get('old', widgetPA.value)
    if(PANew < PAOld):
        # A reduction in PA may decrease max PAB
        PABMax = getPABmax(PANew, PB)
        PAB = min(PAB, PABMax)
        if(PAB != widgetPAB.value):
            widgetPAB.value = PAB        
        if(PABMax != widgetPAB.max):
            widgetPAB.max = PABMax
    if(PANew > PAOld):
        # An increase in PA may increase min PAB and max PAB
        PABMin = getPABmin(PANew, PB)
        PABMax = getPABmax(PANew, PB)
        PAB = min(PAB, PABMax)
        PAB = max(PAB, PABMin)
        if(PABMax != widgetPAB.max):
            widgetPAB.max = PABMax
        if(PAB != widgetPAB.value):
            widgetPAB.value = PAB        
        if(PABMin != widgetPAB.min):
            widgetPAB.min = PABMin

# This function will be called whenever the value of the PB widget is changed
def onWidgetPBUpdate(change):
    PA = widgetPA.value
    PAB = widgetPAB.value
    PBNew = change.get('new', widgetPB.value)
    PBOld = change.get('old', widgetPB.value)
    if(PBNew < PBOld):
        # A reduction in PB may decrease max PAB
        PABMax = getPABmax(PA, PBNew)
        PAB = min(PAB, PABMax)
        if(PAB != widgetPAB.value):
            widgetPAB.value = PAB        
        if(PABMax != widgetPAB.max):
            widgetPAB.max = PABMax
    if(PBNew > PBOld):
        # An increase in PB may increase min PAB and max PAB
        PABMin = getPABmin(PA, PBNew)
        PABMax = getPABmax(PA, PBNew)
        PAB = min(PAB, PABMax)
        PAB = max(PAB, PABMin)
        if(PABMax != widgetPAB.max):
            widgetPAB.max = PABMax
        if(PAB != widgetPAB.value):
            widgetPAB.value = PAB        
        if(PABMin != widgetPAB.min):
            widgetPAB.min = PABMin
        
# This function will be called whenever the value of the PAB widget is changed
def onWidgetPABUpdate(change):
    PA = widgetPA.value
    PB = widgetPB.value
    PABNew = change.get('new', widgetPAB.value)
    PABOld = change.get('old', widgetPAB.value)
    if(PABNew > PABOld):
        # An increase in PAB may increase max PAB
        PABMax = getPABmax(PA, PB)
        PABNew = widgetPAB.value
        PABNew = min(PABNew, PABMax)
        if(PABMax != widgetPAB.max):
            widgetPAB.max = PABMax
        if(PABNew != widgetPAB.value):
            widgetPAB.value = PABNew        
        
def getPABmin(PA, PB):   
    return(max(0,(PA + PB) - 1))
    
def getPABmax(PA, PB):
    return(max(getPABmin(PA, PB),min([PA, PB])))

# Create the UI widgets for interaaction.
widgetPA=widgets.FloatSlider(description='P(A)',min=0,max=1,step=.01,value=.8)
widgetPB=widgets.FloatSlider(description='P(B)',min=0,max=1,step=.01,value=.3) 
widgetPAB=widgets.FloatSlider(description='P(A,B)',min=0,max=1,step=.01,value=.2)

# Configure the event handlers.
widgetPA.observe(onWidgetPAUpdate, 'value')
widgetPB.observe(onWidgetPBUpdate, 'value')
widgetPAB.observe(onWidgetPABUpdate, 'value')

# Initiate user interaction with the UI.
interact(widgetInteraction,
         PA=widgetPA,
         PB=widgetPB,
        PAB=widgetPAB)



interactive(children=(FloatSlider(value=0.8, description='P(A)', max=1.0, step=0.01), FloatSlider(value=0.3, d…

<function __main__.widgetInteraction(PA, PB, PAB)>

In [10]:
min([4,6])


4